# Bemol AI Engineering Challenge

## O desafio consiste em:

* Uma análise do perfil do público de clientes da base de dados;
* Segmentar os perfis identificados com base em características úteis para o time de vendas e marketing;
* Fornecer os dados a partir de uma API utilizada para consultas. Disponível em `/src/api/`.

## Perguntas iniciais:

Talvez algumas dessas não possam ser respondidas pela base de dados, mas é um ponto de partida:

#### Sobre os dados:

* Como separar clientes entre pontuais (que realizam uma ou poucas compras) e clientes recorrentes?
* Será que é possível extrair informações que ajudem a tornar um cliente pontual em um cliente recorrente?
* Dado que existem clientes recorrentes, o que os tornam recorrentes? É possível melhorar esse relacionamento?

#### Sobre o problema:

* Quais métricas são importantes de usar para o problema de segmentação?
   - [Recency, Frequency, Monetary Value (RFM)](https://www.investopedia.com/terms/r/rfm-recency-frequency-monetary-value.asp) Analysis parece ser um bom ponto de partida. Segundo algumas breves pesquisas, queremos entender o quão recente um cliente fez uma compra, o quão frequente um cliente compra e o quanto o cliente gasta em suas compras.
* Que algorítmos são mais indicados para esses casos?

# 1. Exploração Inicial dos Dados

In [1]:
import pandas as pd

Para este desafio, será utilizado o dataset [Online Retail K-means & Hierarchical Clustering](https://www.kaggle.com/hellbuoy/online-retail-customer-clustering) que consiste em dados de transações ocorridas entre 01/12/2010 e 09/12/2011 em lojas baseadas no Reino Unido. A empresa vende presentes para todas as ocasiões e a maioria dos seus clientes são atacadistas.

O motivo para seleção deste foi pela maior quantidade de exemplos existente na base, maior quantidade de atributos e pelos atributos aparentarem ter informações mais relevantes para segmentação de clientes quando comparado a outra base sugerida.

#### Perguntas:

* Se a maioria dos clientes são atacadistas, então a maior parte das vendas são em grandes volumes?

#### 1.1 Explorando atributos

Temos na base de dados os seguintes atributos:

* `InvoiceNo`: ID alfanumérico da nota associada a venda do produto.
* `StockCode`: ID alfanumérico do produto [?]
* `Description`: Descrição textual do produto.
* `Quantity`: Quantidade de peças do produto comprada.
* `InvoiceDate`: Data da compra.
* `UnitPrice`: Preço da unidade do produto.
* `CustomerID`: ID alfanumérico que identifica o cliente.
* `Country`: País onde compra foi realizada [?]

In [2]:
data = pd.read_csv('../data/raw/online-retail-k-means/data.csv')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


#### 1.2 Dados Nulos

Aparentemente temos alguns dados faltando na nossa base de dados, são esses:

* 135080 vendas sem `CustomerID`.
* 1454 vendas sem `Description`.

O atributo `CustomerID` é o mais relevante nesse caso pois estamos explorando informações de clientes e esse trabalho é dificultado sem o ID do cliente.

Duas alternativas podem ser exploradas para resolver a falta de dados:

1. Dado que temos o ID da compra (`InvoiceNo`) talvez seja possível que algumas compras tenham o ID do cliente para alguns produtos e que faltem para outros. Se este caso for verdadeiro, é possível recuperarmos alguns exemplos onde o `CustomerID` não está presente. Um passo similar pode ser realizado para o `Description`.
2. Remover todas as linhas onde `CustomerID` não estão preenchidos. Como o `Description` dos items não são extremamente importantes, é possível trabalharmos sem remover os exemplos onde há falta do `Description`.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Primeiramente vamos explorar a alternativa `1`. Para isso seguiremos os seguintes passos:

1. São obtidas todas as linhas onde `CustomerID` está nulo.
2. Depois, extrai-se todos os `InvoiceNo` onde `CustomerID` está nulo.
3. Busca-se por todos os `InvoiceNo` existentes na base que estão na lista de `CustomerID` e verifica-se se não existe algum com `CustomerID` preenchido.

Após seguir estes passos, é possível saber se existe alguma compra que tem o `CustomerID` preenchido em algum dos items, porém, segundo o resultado da nossa busca, aparentemente não existe esse caso na nossa base de dados, portanto a alternativa `2` será realizada, portanto serão removidos todos os exemplos com `CustomerID` nulos.

O mesmo também ocorre com o atributo `Description`.

In [4]:
# Verificando CustomerID
nan_customer = data[data['CustomerID'].isnull()]
nan_invoices = nan_customer['InvoiceNo'].unique()
boolean_filter = data['InvoiceNo'].isin(nan_invoices)
data[boolean_filter]['CustomerID'].any()

False

In [5]:
# Verificando Description
nan_customer = data[data['Description'].isnull()]
nan_invoices = nan_customer['InvoiceNo'].unique()
boolean_filter = data['InvoiceNo'].isin(nan_invoices)
data[boolean_filter]['Description'].any()

False

In [6]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


#### 1.3 Dados irregulares

Algumas irregularidade nos dados encontradas são:

* Valor de itens zerado.
* Quantidade de itens negativos (-80995?).
* Quantidade muito alta de itens (80995?).

Entretanto esses dados podem não ser irregularidades, mas sim a forma que a empresa identifica compras canceladas e descontos.

É perceptível abaixo que alguns `InvoiceNo` são iniciados com a letra `C` e que são seguidos de compras em quantidades negativas, o que dá a entender que são compras canceladas.

#### Observação:

* Este tópico de compras com desconto pode ajudar a encontrar padrões em relação ao tipo de produtos que os clientes gostam de ter descontos e se isso pode aumentar o número de vendas.
* Apesar disso, a falta de descrição dos produtos para categorização e agrupamento dificulta uma análise mais profunda desse aspecto nesta base de dados.

In [7]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [8]:
# Devolução?
data[data['Quantity'] < 0].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,01-12-2010 09:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,01-12-2010 09:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,01-12-2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,01-12-2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,01-12-2010 10:24,0.29,17548.0,United Kingdom


In [9]:
# Descontos?
data[data['UnitPrice'] <= 0].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9302,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,05-12-2010 14:02,0.0,12647.0,Germany
33576,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,16-12-2010 14:36,0.0,16560.0,United Kingdom
40089,539722,22423,REGENCY CAKESTAND 3 TIER,10,21-12-2010 13:45,0.0,14911.0,EIRE
47068,540372,22090,PAPER BUNTING RETROSPOT,24,06-01-2011 16:41,0.0,13081.0,United Kingdom
47070,540372,22553,PLASTERS IN TIN SKULLS,24,06-01-2011 16:41,0.0,13081.0,United Kingdom


In [10]:
# Compra cancelada?
data[data['Quantity'].isin([80995, -80995])]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,09-12-2011 09:15,2.08,16446.0,United Kingdom
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,09-12-2011 09:27,2.08,16446.0,United Kingdom


# 2. Exploração de Dados de Clientes

Agora que demos uma olhada geral no dataset e de algumas características específicas dos dados, exploraremos as informações relacionadas aos clientes com o objetivo de segmentá-los de forma útil para o time de marketing e de vendas.

#### 2.1 Segmentação por localização:

Dado que localidade é uma característica que pode influenciar nas características do cliente, a primeira segmentação sugerida é a partir dos dados de localização dos clientes.

Clientes que moram em países diferentes tem necessidades diferentes e, por conta da distância geográfica e relações diplomática entre os países, a empresa pode  fornecer atendimento distinto entre clientes de localidades distintas.

In [44]:
# Quantidade de vendas por país
sell_by_country = data.drop_duplicates('InvoiceNo')['Country'].value_counts()
sell_by_country[:10]

United Kingdom    19857
Germany             603
France              458
EIRE                319
Belgium             119
Spain               105
Netherlands         101
Switzerland          71
Portugal             70
Australia            69
Name: Country, dtype: int64

In [45]:
# Quantidade de vendas por cliente
sell_by_client = data.drop_duplicates('CustomerID')['Country'].value_counts()
sell_by_client[:10]

United Kingdom    3950
Germany             95
France              87
Spain               29
Belgium             24
Switzerland         20
Portugal            19
Italy               15
Finland             12
Norway              10
Name: Country, dtype: int64

In [43]:
client_by_country_ratio = sell_by_country / sell_by_client
client_by_country_ratio

Australia                 7.666667
Austria                   2.111111
Bahrain                   1.000000
Belgium                   4.958333
Brazil                    1.000000
Canada                    1.500000
Channel Islands           3.666667
Cyprus                    2.857143
Czech Republic            5.000000
Denmark                   2.625000
EIRE                    106.333333
European Community        5.000000
Finland                   4.000000
France                    5.264368
Germany                   6.347368
Greece                    1.500000
Iceland                   7.000000
Israel                    1.500000
Italy                     3.666667
Japan                     3.500000
Lebanon                   1.000000
Lithuania                 4.000000
Malta                     5.000000
Netherlands              11.222222
Norway                    4.000000
Poland                    4.000000
Portugal                  3.684211
RSA                       1.000000
Saudi Arabia        